In [ ]:
import torch

### The cross-correlation operator

In [ ]:
def corr2d(X, K):
    H, W = # insert your code here
    h, w = # insert your code here
    Y = # insert your code here
    # insert your code here
    return Y

In [ ]:
# test your function

### Convolutional Layers

A convolutional layer cross-correlates the input and kernels and adds a scalar bias to produce an output.
The parameters of the convolutional layer are precisely the values that constitute the kernel and the scalar
bias. When training the models based on convolutional layers, we typically initialize the kernels randomly,
just as we would with a fully-connected layer.

In [ ]:
# # insert your code here

In [ ]:
# let's test

### Object Detection in Images

Let’s look at a simple application of a convolutional layer: detecting the edge of an object in an image by
finding the location of the pixel change. First, we construct an 'image' of 6 x 8 pixels. The middle four
columns are black (0) and the rest are white (1).

In [ ]:
# create X

Next, we construct a kernel K with a height of 1 and width of 2. When we perform the cross-correlation
operation with the input, if the horizontally adjacent elements are the same, the output is 0. Otherwise, the
output is non-zero.

In [ ]:
# create Kernel

Enter X and our designed kernel K to perform the cross-correlation operations. As you can see, we will detect
1 for the edge from white to black and -1 for the edge from black to white. The rest of the outputs are 0.

In [ ]:
# check output

Let’s apply the kernel to the transposed image.

In [ ]:
# insert your code here

### Learning a Kernel

Designing an edge detector by finite differences [1, -1] is neat if we know this is precisely what we are
looking for. However, as we look at larger kernels, and consider successive layers of convolutions, it might
be impossible to specify precisely what each filter should be doing manually.

Now let’s see whether we can learn the kernel that generated Y from X by looking at the (input, output)
pairs only. We first construct a convolutional layer and initialize its kernel as a random array. Next, in
each iteration, we will use the squared error to compare Y and the output of the convolutional layer, then
calculate the gradient to update the weight.

In [ ]:
# insert your code here

As you can see, the error has dropped to a small value after 10 iterations. Now we will take a look at the kernel array we learned.

In [ ]:
# check the learned weights

Indeed, the learned kernel array is remarkably close to the kernel array K we defined earlier.